#### Ridge Regression (L2)

- Loss = MSE + α * Σ(coefficients²)

In [28]:
from sklearn.datasets import fetch_california_housing
from sklearn.preprocessing import StandardScaler
from sklearn.linear_model import Ridge, Lasso, ElasticNet, LinearRegression
from sklearn.model_selection import cross_val_score, train_test_split
from sklearn.metrics import r2_score, mean_squared_error
import numpy as np
import matplotlib.pyplot as plt

In [ ]:
# Load data
housing = fetch_california_housing()
x, y = housing.data, housing.target

# Split data
x_train, x_test, y_train, y_test = train_test_split(x, y, test_size=0.25, random_state=42)

# Scale features
scaler = StandardScaler()
x_train_scaled = scaler.fit_transform(x_train)
x_test_scaled = scaler.transform(x_test)

In [25]:
ridge = Ridge(alpha=0.9)
ridge.fit(x_train_scaled, y_train)

y_pred = ridge.predict(x_test_scaled)

rmse = np.sqrt(mean_squared_error(y_test, y_pred))
r2 = r2_score(y_test, y_pred)
print(f"Ridge - RMSE: {rmse:.2f}, R²: {r2:.2f}")
print("\nRidge coefficients (smaller):", ridge.coef_)

Ridge - RMSE: 0.74, R²: 0.59

Ridge coefficients (smaller): [ 0.85204915  0.1207297  -0.30189935  0.34835155 -0.00162016 -0.04116712
 -0.89242736 -0.8671104 ]


In [17]:
# Cross-validation to find best alpha
alphas = [0.01, 0.1, 0.5, 1.0, 10.0, 100.0]
scores = []

for alpha in alphas:
    ridge = Ridge(alpha=alpha)
    score = cross_val_score(ridge, x_train_scaled, y_train, cv=5, scoring='neg_mean_squared_error')
    scores.append(-score.mean())
    
best_alpha = alphas[np.argmin(scores)]
print(f"\nBest alpha: {best_alpha}")


Best alpha: 10.0


#### Lasso Regression (L1)
- Loss = MSE + α * Σ|coefficients|

In [24]:
lasso = Lasso(alpha=0.2)
lasso.fit(x_train_scaled, y_train)

y_pred = lasso.predict(x_test_scaled)

rmse = np.sqrt(mean_squared_error(y_test, y_pred))
r2 = r2_score(y_test, y_pred)
print(f"Ridge - RMSE: {rmse:.2f}, R²: {r2:.2f}")

Ridge - RMSE: 0.86, R²: 0.44


In [23]:
# Feature selection (coefficients set to zero)
feature_names = housing.feature_names
print("\nLasso coefficients:")
for name, coef in zip(feature_names, lasso.coef_):
    if abs(coef) > 0.001:
        print(f"{name}: {coef:.3f}")
    else:
        print(f"{name}: 0.000 (eliminated)")


Lasso coefficients:
MedInc: 0.596
HouseAge: 0.000 (eliminated)
AveRooms: 0.000 (eliminated)
AveBedrms: 0.000 (eliminated)
Population: 0.000 (eliminated)
AveOccup: 0.000 (eliminated)
Latitude: 0.000 (eliminated)
Longitude: 0.000 (eliminated)


#### Elastic Net
- Loss = MSE + α * (λ * L1 + (1-λ) * L2)

In [ ]:
elastic = ElasticNet(alpha=0.2, l1_ratio=0.5)
elastic.fit(x_train_scaled, y_train)

y_pred = elastic.predict(x_test_scaled)

rmse = np.sqrt(mean_squared_error(y_test, y_pred))
r2 = r2_score(y_test, y_pred)
print(f"Elastic Net - RMSE: {rmse:.2f}, R²: {r2:.2f}")

Elastic Net - RMSE: 0.83, R²: 0.48


#### Comparison

In [29]:
models = {
    'Linear' : LinearRegression(),
    'Ridge' : Ridge(alpha=1.0),
    'Lasso' : Lasso(alpha=0.1),
    'Elastic Net' : ElasticNet(alpha=0.1, l1_ratio=0.5)
}

results = {}
for name, model in models.items():
    model.fit(x_train_scaled, y_train)
    y_pred = model.predict(x_test_scaled)
    rmse = np.sqrt(mean_squared_error(y_test, y_pred))
    r2 = r2_score(y_test, y_pred)
    results[name] = {'RMSE' : rmse, 'R²': r2}
    
for name, metrics in results.items():
    print(f"{name}: RMSE={metrics['RMSE']:.2f}, R²={metrics['R²']:.3f}")

Linear: RMSE=0.74, R²=0.591
Ridge: RMSE=0.74, R²=0.591
Lasso: RMSE=0.82, R²=0.492
Elastic Net: RMSE=0.79, R²=0.526
